In [ ]:
import zipfile
import glob
import pandas as pd
import numpy as np

from argparse import ArgumentParser
from argparse import RawDescriptionHelpFormatter
import sys
import os
if  not './' in sys.path:
    sys.path.append('./')
if  not '../' in sys.path:
    sys.path.append('../')

from barchartacs import build_db
from barchartacs import db_info
import datetime
import io
from tqdm import tqdm,tqdm_notebook
from barchartacs import pg_pandas as pg
import urllib.request, urllib.parse, urllib.error
import ipdb
import importlib
# importlib.reload(db_info)

In [ ]:
WRITE_TO_POSTGRES = False
CONTRACT_LIST = ['ES']#['CL','CB']
STRIKE_DIVISOR = 1
csv_temp_path = './temp_folder/df_all_temp.csv'

In [ ]:
def do_request(url,dict_headers=None):
    '''
    Example:
    1. get google page
        text_of_webpage =  do_request('https://www.google.com')
    2. get text file from barchart daily 
        barchart_daily_text_file = do_request('http://acs.barchart.com/mri/data/opv07059.csv',dict_headers={"Authorization": "Basic myauthcode"})

    '''
    if dict_headers is None:
        req = urllib.request.Request(url)
    else:
        req = urllib.request.Request(url, headers=dict_headers)
    f = urllib.request.urlopen(req)
    alines = f.read()#.decode('utf-8')
    return alines

In [ ]:
def build_daily(commod_code_list,yyyymmdd=None,barchart_auth_code=None,text_file_path=None):
    '''
    commod_code_list: like ['ES'] or ['CL','CB']
    yyyymmdd: like 20190705.  It must be a date in the current month
    Example:
    df_this_day = build_daily(['ES'],20190705)
    '''
    tfp = './temp_folder/opv.txt' if text_file_path is None else text_file_path
    ymd = yyyymmdd
    if ymd is None:
        # get yesterday
        ymd = int((datetime.datetime.now()-datetime.timedelta(1)).strftime('%Y%m%d'))
    bac = barchart_auth_code
    if bac is None:
        bac = open('./temp_folder/barchart_authcode.txt','r').read()
    y = str(ymd)[3]
    mm = str(ymd)[4:6]
    dd = str(ymd)[6:8]
    opv = 'opv' + mm + dd + y
    url = f'http://acs.barchart.com/mri/data/{opv}.csv'
    dict_header = {"Authorization": f"Basic {bac}"}
    opvtxt = do_request(url,dict_headers=dict_header)
    opvtxt = opvtxt.decode('utf-8')
    open(tfp,'w').write(opvtxt)
    builder = build_db.BuildDb(None,strike_divisor=1,
                           contract_list=commod_code_list,write_to_database=False)
    dft = builder.build_options_pg_from_csvs(tfp)
    return dft


### Single day test for yesterday
uncomment the line below to test for a single day

In [ ]:
# df_es = build_daily(commod_code_list=['CL','CB'])
# df_es.head()

### Execute a range of days

In [ ]:
df_all = None
ipdb.set_trace()
year = 2019
month = 7
day_list = [1,2,3,5,8,9]
for day in tqdm_notebook(day_list):
    yyyymmdd = year*100*100 + month*100 + day
    print(f'executing yyyymmdd {yyyymmdd} at {datetime.datetime.now()}')
    try:
        df_temp = build_daily(commod_code_list=CONTRACT_LIST,yyyymmdd=yyyymmdd)
        if df_all is None:
            df_all = df_temp.copy()
        else:
            df_all = df_all.append(df_temp)
            df_all.index = list(range(len(df_all)))
    except Exception as e:
        bdb.logger.warn(f'ERROR MAIN LOOP: {str(e)}')
    
# NOW WRITE THIS DATA FOR THIS YEAR
df_all.to_csv(csv_temp_path,index=False)
if WRITE_TO_POSTGRES:
    bdb.logger.info(f'MAIN LOOP: writing data for year {yyyy} to database')
    abspath = os.path.abspath(csv_temp_path)
    bdb.pga.exec_sql_raw(f"COPY sec_schema.options_table FROM '{abspath}' DELIMITER ',' CSV HEADER")    

In [ ]:
print(df_all.shape)
print(sorted(df_all.symbol.unique()))
print(df_all.settle_date.min(),df_all.settle_date.max())
df_all.tail()

### END